In [ ]:
EJ56 = DstApi('EJ56')

# Landsdele codes (01-11)
landsdele_codes = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']

# Property types: Enfamiliehuse, Sommerhuse, Ejerlejligheder
ejendom_codes = ['0111', '0801', '2103']

# Download data
params = {
    'table': 'EJ56',
    'format': 'BULK',       
    'lang': 'da',
    'variables': [
        {'code': 'OMRÅDE', 'values': landsdele_codes},
        {'code': 'EJENDOMSKATE', 'values': ejendom_codes},
        {'code': 'TAL', 'values': ['100']},  # 100 = Indeks
        {'code': 'Tid', 'values': ['*']}
    ]
}

df = EJ56.get_data(params=params)

# Clean data: replace '..' with NaN
df = df.replace('..', np.nan)
df['INDHOLD'] = df['INDHOLD'].str.replace(',', '.').astype(float)
df = df.dropna(subset=['INDHOLD'])

# Rename columns
df = df.rename(columns={
    'INDHOLD': 'Indeks', 
    'TID': 'Kvartal', 
    'OMRÅDE': 'Landsdel', 
    'EJENDOMSKATE': 'Ejendomskategori', 
    'TAL': 'Enhed'
})

# Convert quarterly time
df['År'] = df['Kvartal'].str[:4].astype(int)
df['Kvartal_nr'] = df['Kvartal'].str[-1].astype(int)
df['Kvartal_periode'] = pd.PeriodIndex.from_fields(year=df['År'], quarter=df['Kvartal_nr'], freq='Q')

# Sort data
df = df.sort_values(['Landsdel', 'Ejendomskategori', 'År', 'Kvartal_nr']).reset_index(drop=True)

# FILTER: Keep only landsdele that have data starting from 1992Q1
# For each property type, identify which landsdele start from 1992Q1
earliest_quarter = df['Kvartal'].min()  # Should be 1992K1


landsdele_to_keep = []
for ejendom in df['Ejendomskategori'].unique():
    df_ejendom = df[df['Ejendomskategori'] == ejendom]
    for landsdel in df_ejendom['Landsdel'].unique():
        df_subset = df_ejendom[df_ejendom['Landsdel'] == landsdel].sort_values(['År', 'Kvartal_nr'])
        if len(df_subset) > 0:
            first_quarter = df_subset['Kvartal'].iloc[0]
            # Only keep if starts from earliest quarter (1992K1)
            if first_quarter == earliest_quarter:
                landsdele_to_keep.append((landsdel, ejendom))


# Filter dataframe to keep only complete series
mask_keep = df.apply(lambda row: (row['Landsdel'], row['Ejendomskategori']) in landsdele_to_keep, axis=1)
df = df[mask_keep].reset_index(drop=True)

# REINDEX TO 1992Q1 = 100
# For each landsdel and ejendomskategori, normalize to first observation (1992Q1)
df['Indeks_1992'] = np.nan

for landsdel in df['Landsdel'].unique():
    for ejendom in df['Ejendomskategori'].unique():
        mask = (df['Landsdel'] == landsdel) & (df['Ejendomskategori'] == ejendom)
        df_subset = df[mask].sort_values(['År', 'Kvartal_nr'])
        
        if len(df_subset) > 0:
            # Get the first (earliest) index value
            base_value = df_subset['Indeks'].iloc[0]
            # Normalize: (current / base) * 100
            df.loc[mask, 'Indeks_1992'] = (df.loc[mask, 'Indeks'] / base_value) * 100



In [ ]:
# Question 1.a: Analyze property price index development across landsdele
# Using reindexed data where 1992Q1 = 100

# Create time series plot for each property type across all landsdele
# Arrange plots vertically for better readability
fig, axes = plt.subplots(3, 1, figsize=(12, 14))

property_types = df['Ejendomskategori'].unique()

for idx, prop_type in enumerate(property_types):
    ax = axes[idx]
    df_prop = df[df['Ejendomskategori'] == prop_type]
    
    # Plot each landsdel
    for landsdel in sorted(df_prop['Landsdel'].unique()):
        df_plot = df_prop[df_prop['Landsdel'] == landsdel].sort_values(['År', 'Kvartal_nr'])
        # Convert to datetime for proper plotting
        df_plot['date'] = df_plot['Kvartal_periode'].dt.to_timestamp()
        ax.plot(df_plot['date'], df_plot['Indeks_1992'], label=landsdel, alpha=0.7)
    
    ax.set_title(f'{prop_type}')
    ax.set_xlabel('År')
    ax.set_ylabel('Prisindeks (1992Q1=100)')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.axhline(y=100, color='black', linestyle='--', alpha=0.5, linewidth=0.8)
    
    # Format x-axis to show years nicely
    import matplotlib.dates as mdates
    ax.xaxis.set_major_locator(mdates.YearLocator(5))  # Major tick every 5 years
    ax.xaxis.set_minor_locator(mdates.YearLocator(1))   # Minor tick every year
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()
